### import libraries

In [ ]:
from opensearchpy import OpenSearch
from dotenv import load_dotenv
from pprint import pprint
import re

In [3]:
#load the enviornment variables from .env
load_dotenv()
from os import getenv

USERNAME = getenv("OPENSEARCH_ADMIN_USER")
PASSWORD = getenv("OPENSEARCH_INITIAL_ADMIN_PASSWORD")

### connect to host

In [12]:
host = 'localhost'
port = 19200
auth = (USERNAME, PASSWORD) # For testing only. Don't store credentials in code.
# ca_certs_path = '/full/path/to/root-ca.pem' # Provide a CA bundle if you use intermediate CAs with your root CA.

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = auth,
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
)
client.info()

{'name': 'opensearch-node1',
 'cluster_name': 'opensearch-cluster',
 'cluster_uuid': 'GAWfTRd5RW-CucmhKN6hVA',
 'version': {'distribution': 'opensearch',
  'number': '3.0.0',
  'build_type': 'tar',
  'build_hash': 'dc4efa821904cc2d7ea7ef61c0f577d3fc0d8be9',
  'build_date': '2025-05-03T06:25:26.379676844Z',
  'build_snapshot': False,
  'lucene_version': '10.1.0',
  'minimum_wire_compatibility_version': '2.19.0',
  'minimum_index_compatibility_version': '2.0.0'},
 'tagline': 'The OpenSearch Project: https://opensearch.org/'}

### check my index

In [16]:
response = client.search(
    index="document_v4",
    body={
        "_source": ["content"],
        "query": {
            "match_all": {}
        }
    }
)

response

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 3, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'document_v4',
    '_id': '1234',
    '_score': 1.0,
    '_source': {'content': 'There once was a racecar driver that was super fast'}},
   {'_index': 'document_v4',
    '_id': '1235',
    '_score': 1.0,
    '_source': {'content': 'The golf driver used by tiger woods is the TaylorMade Qi10 LS prototype'}},
   {'_index': 'document_v4',
    '_id': '1236',
    '_score': 1.0,
    '_source': {'content': 'Some may say that supercar drivers dont really mind risk'}}]}}

In [21]:
#delete index
response = client.indices.delete("test-document-01", ignore_unavailable=True)
pprint(response)
# create index
response = client.indices.create("test-document-01")
pprint(response)

{'acknowledged': True}
{'acknowledged': True, 'index': 'test-document-01', 'shards_acknowledged': True}


### confirm the get method works

In [23]:
item_a = client.get(index="document_v4", id="1236")
pprint(item_a)

{'_id': '1236',
 '_index': 'document_v4',
 '_primary_term': 1,
 '_seq_no': 2,
 '_source': {'content': 'Some may say that supercar drivers dont really mind '
                        'risk',
             'content_embedding': {'##ability': 0.0043414626,
                                   '##away': 0.09657053,
                                   '##bility': 0.683933,
                                   '##car': 1.8049147,
                                   '##carriage': 0.18450333,
                                   '##cars': 0.7819286,
                                   '##erving': 0.741623,
                                   '##ever': 0.16756783,
                                   '##free': 0.16112545,
                                   '##hip': 0.08930432,
                                   '##moto': 0.70712644,
                                   '##oot': 0.37504345,
                                   '##ruck': 0.124648124,
                                   '##tical': 0.41809833,
       

In [25]:
item_a['_source']['content']

'Some may say that supercar drivers dont really mind risk'

### testing filter regex

In [36]:
import re
import json

In [ ]:
def extract_filters(query):
    filters= []
    category_regex = r'category:([^\s]+)\s*'
    matches = re.search(category_regex, query)
    if matches:
        filters.append({
            'term': {
                'category.keyword': {
                    'value': matches.group(1)
                }
            }
        })

        #remove the category filter from the query
        query = re.sub(category_regex, '', query).strip()

    #year filter
    year_regex = r'year:([^\s]+)\s*'
    matches = re.search(year_regex, query)
    if matches:
        filters.append({
            'range': {
                'updated_at': {
                    'gte': f'{matches.group(1)}||/y',
                    'lte': f'{matches.group(1)}||/y',
                }
            },
        })
        #remove the year filter from the query
        query = re.sub(year_regex, '', query).strip()

    return {'filter': filters}, query

def handle_search():
    filters, parsed_query = extract_filters(query)
    if parsed_query:
        search_query = {
            'must': {
                'multi_match': {
                    'query': parsed_query,
                    'fields': ['name', 'summary', 'content']
                }
            }
        }
    else:
        search_query = {
            'must': {
                'match_all': {}
            }
        }
    results = client.search(
        index="my_documents",
        body={
            'query': {
                'bool': {
                    **search_query,
                    **filters
                }
            },
            'aggs': {
                'category-agg': {
                    'terms': {
                        'field': 'category.keyword',
                    }
                },
                'year-agg': {
                    'date_histogram': {
                        'field': 'updated_at',
                        'calendar_interval': 'year',
                        'format': 'yyyy',
                    },
                },
            },
            'size': 5,
        }
    )
    return results

In [57]:
query = 'work from home category:sharepoint'
results = handle_search()
results

{'took': 5,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 7, 'relation': 'eq'},
  'max_score': 1.0137551,
  'hits': [{'_index': 'my_documents',
    '_id': 'lkp6SJcBpHtcKjIoZdp_',
    '_score': 1.0137551,
    '_source': {'content': "Purpose\n\nThe purpose of this vacation policy is to outline the guidelines and procedures for requesting and taking time off from work for personal and leisure purposes. This policy aims to promote a healthy work-life balance and encourage employees to take time to rest and recharge.\nScope\n\nThis policy applies to all full-time and part-time employees who have completed their probationary period.\nVacation Accrual\n\nFull-time employees accrue vacation time at a rate of [X hours] per month, equivalent to [Y days] per year. Part-time employees accrue vacation time on a pro-rata basis, calculated according to their scheduled work hours.\n\nVacation time will begin to accrue from the f

In [49]:
results

{'took': 5,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 1.0790452,
  'hits': [{'_index': 'my_documents',
    '_id': 'lUp6SJcBpHtcKjIoZdp_',
    '_score': 1.0790452,
    '_source': {'summary': "This sales strategy document outlines objectives, focus areas, and action plans for our tech company's sales operations in fiscal year 2024. Our primary goal is to increase revenue, expand market share, and strengthen customer relationships in our target markets. Focus areas include targeting new markets, segmenting customers, enhancing",
     'name': 'Fy2024 Company Sales Strategy',
     'content': "Executive Summary:\nThis sales strategy document outlines the key objectives, focus areas, and action plans for our tech company's sales operations in fiscal year 2024. Our primary goal is to increase revenue, expand market share, and strengthen customer relationships in our target markets

In [35]:
print(extract_filters("category:news Summary of the latest"))

query: Summary of the latest
({'filter': [{'term': {'category.keyword': {'value': 'news'}}}]}, 'Summary of the latest')
